[View in Colaboratory](https://colab.research.google.com/github/pasumarthi/DeepLearning/blob/master/Suresh_CIFAR10_1505.ipynb)

In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

Using TensorFlow backend.


Increase learning rate fast for 5 iterations and then reduce for every 5 to 0.1 
Decrease batch size

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
batch_size = 64
num_classes = 10
epochs = 50
l = 12
num_filter = 12
compression = 0.6
dropout_rate = 0.5

In [0]:
 #x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
print(img_height, img_width, channel)

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

32 32 3


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12 , dropout_rate = 0.3):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 12
l = 12
input = Input(shape=(img_height, img_width, channel,))
#First_Conv2D = Conv2D(num_filter, (7,7), use_bias=False)(input)
#Second_Conv2D = Conv2D(num_filter, (3,3), use_bias=False)(First_Conv2D)
First_Conv2D = Conv2D(num_filter*2, (3,3),use_bias=False,padding='same')(input)
Second_Con2D =Conv2D(num_filter*4,(7,1),use_bias=False ,padding='same')(First_Conv2D)
Third_Con2D=Conv2D(num_filter*4, (1,7), use_bias = False, padding = 'same')(Second_Con2D)
Forth_Con2D=Conv2D(num_filter, (1,1), use_bias = False, padding = 'same')(Third_Con2D)


First_Block = add_denseblock(Forth_Con2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Forth_Block = add_denseblock(input, num_filter, dropout_rate)
Forth_Transition = add_transition(Forth_Block, num_filter, dropout_rate)

Fifth_Block = add_denseblock(Forth_Transition, num_filter, dropout_rate)
Fifth_Transition = add_transition(Second_Block, num_filter, dropout_rate)

a


Last_Block = add_denseblock(Fifth_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

In [0]:
#opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])ion(Second_Block, num_filter, dropout_rate)

In [0]:
from keras.callbacks import LearningRateScheduler
def lr_schedule(epoch):
    lrate = 0.1
    if epoch > 20:
        lrate = 0.03
    elif epoch > 35:
        lrate = 0.02        
    return lrate

In [0]:
from keras.callbacks import ModelCheckpoint 
ckpt      = ModelCheckpoint('./ckpt.h5', save_best_only=False, mode='auto', period=10)

In [0]:
# set data augmentation


    #print('Using real-time data augmentation.')
    #datagen   = ImageDataGenerator(horizontal_flip=True,width_shift_range=0.125,height_shift_range=0.125,fill_mode='constant',cval=0.)

    #datagen.fit(x_train)
  

In [0]:
#data augmentation
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    #featurewise_std_normalization=True,
    rotation_range=15,
    zca_whitening=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
    )
datagen.fit(x_train)
#datagen.fit(x_test)

In [0]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,\
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule,ckpt)])

In [0]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model2.h5")
print("Saved model to disk")

In [0]:
from google.colab import files

files.download('DNST_model2.h5')

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = 'REPLACE_WITH_YOUR_FILE_ID'
downloaded = drive.CreateFile({'id': file_id})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))